# This is my first Jupyter Notebook!

In [1]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
import sys
sys.path.append("..")

import config

In [2]:
domain = "10YBE----------2"

types = pd.read_csv("../PsrType.csv", sep=";")
types.set_index("Code", inplace=True)

dates = range(2015, 2023)
#generationACT_df = pd.DataFrame([], columns = ["type", "quantity", "start", "end"])
generationACT_df = pd.DataFrame([], columns = ["datetime", "type", "generation"])

FileNotFoundError: [Errno 2] No such file or directory: '../PsrType.csv'

In [3]:
print("Hello World!")

Hello World!


In [5]:
def get_response(year, BZ):
    
    """
    This function takes in a year and a bidding zone.
    It calls the API, and returns the API response.
    This can be fed to the data format function.
    """
    
    startdate = str(year) + "01010000"
    enddate = str(year) + "12312300"
    domain = BZ
    
    try:
        response = requests.get("https://web-api.tp.entsoe.eu/api",
                        params = {"securityToken": config.security_token,
                                "documentType": "A75",
                                "processType": "A16",
                                #"psrType": "B02", can put the type of generator
                                "in_Domain": domain,
                                "periodStart": startdate,
                                "periodEnd": enddate})
    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    return response

In [10]:
def parse_data(resp):
    """
    This function takes in a response from the API.
    It returns a pandas dataframe with variables start and price. 
    Price is a number, and start is a datetime variable.
    The index starts at 0 and goes until the end.
    This function is to be executed per year, per bidding zone.
    Resulting dataframes can then later be combined.
    """
    response = resp

    root = ET.fromstring(response.content)
    timeseries = root.findall("./{urn:iec62325.351:tc57wg16:451-6:generationloaddocument:3:0}TimeSeries")

    df_cols = ["type", "start", "position", "quantity"]
    df_rows = []

    for ts in timeseries:
        data = ts[7]
        for obs in data[2:]:
            df_rows.append({
                "type": ts[6][0].text,
                "start": data[0][0].text,
                "position": obs[0].text,
                "quantity": obs[1].text
        })

    combined_df = pd.DataFrame(df_rows, columns = df_cols)
    combined_df["type"] = types.loc[combined_df["type"], "Meaning"].values

    #print(f"Resolution {data[1].text}")

    combined_df["start"] = combined_df["start"].astype("datetime64")
    combined_df["position"] = pd.to_timedelta(combined_df["position"].astype("int"), unit="h")

    out_df = pd.DataFrame([], columns = ["datetime", "type", "generation"])
    out_df["datetime"] = combined_df["start"] + combined_df["position"] - pd.to_timedelta(1, unit="h")
    out_df["type"] = combined_df["type"]
    out_df["generation"] = combined_df["quantity"]

    return out_df

In [11]:
for year in tqdm(dates):
    resp = get_response(year, domain)
    out_df = parse_data(resp)
    generationACT_df = pd.concat([generationACT_df, out_df], axis=0, ignore_index=True)

100%|█████████████████████████████████████████████| 8/8 [07:13<00:00, 54.14s/it]


In [12]:
generationACT_df.to_csv("../data/temp/generationACT.csv", index=False)